In [2]:
# %matplolib inline
import pandas as pd 
from matplotlib import pyplot as plt 
import os

In [3]:
prefix_path = "../data/jane-street-market-prediction/"
feature_path = prefix_path + "features.csv"
train_path = prefix_path + "train.csv"
example_test_path = prefix_path + "example_test.csv" 

In [8]:
# 股票特征的分布
feature = pd.read_csv(feature_path)


In [42]:
# 读取训练集
examples = pd.read_csv(train_path,nrows=500)

In [44]:
# 统计特征的缺失值，对缺失值超过10%的特征进行删除（暂定）
cnt = len(train)
feature_percent = examples.isnull().sum()/cnt
feature_percent = feature_percent.apply(lambda x:x<0.1)
train_feature = []
for col in feature_percent.keys():
    if col not in ["date","weight","ts_id","resp_1", "resp_2", "resp_3","resp_4", "resp"] and feature_percent[col]:
        train_feature.append(col)

In [52]:
# 构建训练集、验证集、标签
from sklearn.model_selection import train_test_split

examples["label"] = (examples['resp']+examples["resp_1"]+examples["resp_2"]+examples["resp_3"]+examples["resp_4"])/5
train = examples[train_feature]
label = examples["label"]
train_set,eval_set,train_label,eval_label = train_test_split(train,label,test_size=0.2)
print(len(train_set))

400


In [53]:
# 构建线性回归模型
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(train_set, train_label)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [55]:
# 预测
train_pred = linreg.predict(train_set)
eval_pred = linreg.predict(eval_set)

# 衡量模型效果
from sklearn import metrics
import numpy as np
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(train_label, train_pred))
print("MSE:",metrics.mean_squared_error(eval_label, eval_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_label, train_pred)))
print("RMSE:",np.sqrt(metrics.mean_squared_error(eval_label, eval_pred)))

MSE: 0.0002447627632323372
MSE: 0.0007141929094557836
RMSE: 0.01564489575651871
RMSE: 0.02672438791545624


In [ ]:
# 提交预测
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in iter_test:
    select_df = test_df[select_feature]
    reward = linreg.predict(select_df)
    if reward > 0:
        sample_prediction_df.action = 1
    else:
        sample_prediction_df.action = 0
    env.predict(sample_prediction_df)